<a href="https://colab.research.google.com/github/torrhen/pytorch/blob/main/05_pytorch_script_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
import os
os.makedirs('script_mode', exist_ok=True)

In [12]:
import os
import zipfile # file extracting
from pathlib import Path # file management
import requests # HTTP requests

# store file paths
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

# create the image pth folder if it does not exist
if image_path.is_dir():
  print(f"{image_path} already exists.")
else:
  image_path.mkdir(parents=True, exist_ok=True)

# download image zip file from GitHub
with open(data_path / "pizza_steak_sushi.zip", 'wb') as f:
  response = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(response.content)

# extract images from image zip file
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", 'r') as z:
  z.extractall(image_path)

# delete download image zip file - no longer required
os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi already exists.


In [13]:
# training and test folders
train_folder = image_path / 'train'
test_folder = image_path / 'test'

In [14]:
# write the following code to a new file data.py
%%writefile script_mode/data.py

import os
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

def create_dataloaders(train_dir: str, test_dir: str, transform: transforms.Compose, batch_size: int, num_workers: int = os.cpu_count()):
  # create datasets from image file folder and apply transform
  train_data = datasets.ImageFolder(root=train_dir, transform=transform, target_transform=None)
  test_data = datasets.ImageFolder(root=test_dir, transform=transform, target_transform=None)

  # divide the datasets into batches and shuffle if applicable
  train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return train_loader, test_loader, train_data.classes

Overwriting script_mode/data.py


In [15]:
# write the following code to a new file model.py
%%writefile script_mode/model.py
import torch
import torch.nn as nn

class TinyVGG(nn.Module):
  def __init__(self, input_size, hidden_units, output_size):
    super().__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_size, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=input_size, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features=output_size)
    )

  def foward(self, x):
    return self.fc(self.block2(self.block1(x)))




Overwriting script_mode/model.py


In [16]:
import torch
from script_mode import model

torch.manual_seed(42)
model1 = model.TinyVGG(3, 10, 3).to(device)